In [1]:
# importing libraries
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from datetime import datetime as dt 
from pathlib import Path
import streamlit_keplergl
from numerize import numerize
from PIL import Image
import os
import streamlit as st

C:\Users\nikik\anaconda3\envs\citibike2022\Lib\site-packages\keplergl\keplergl.py:13: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_string


In [2]:
DATA_DIR = Path(r"C:\Users\nikik\citibike_tripdata\citibike-dashboard\2022-citibike-tripdata")

In [3]:
csv_files = list(DATA_DIR.rglob("*.csv"))
len(csv_files)

38

In [4]:
pd.read_csv(csv_files[0], nrows=5).columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'date', 'tavg_c', 'station'],
      dtype='object')

In [5]:
wanted_cols = [
    "started_at",
    "start_station_name",
    "start_lat",
    "start_lng",
    "end_station_name",
    "end_lat",
    "end_lng",
    "member_casual"
]
wanted_set = set(wanted_cols)

required_cols = {"started_at", "start_station_name", "end_station_name"}


In [6]:
np.random.seed(32)

CHUNKSIZE = 100_000
SAMPLE_RATE = 0.00345
samples = []
skipped = []

for f in csv_files:
   
    cols = set(pd.read_csv(f, nrows=0).columns)

    if not required_cols.issubset(cols):
        skipped.append(f.name)
        continue

    for chunk in pd.read_csv(
        f,
        usecols=lambda c: c in wanted_set,   # only read columns that exist
        chunksize=CHUNKSIZE,
        low_memory=False
    ):
        chunk = chunk.dropna(subset=["start_station_name", "end_station_name", "started_at"])
        if chunk.empty:
            continue

        mask = np.random.rand(len(chunk)) <= SAMPLE_RATE
        sampled = chunk.loc[mask]
        if not sampled.empty:
            samples.append(sampled)

small = pd.concat(samples, ignore_index=True)

print("Sample shape:", small.shape)
print("Skipped files:", len(skipped))
skipped[:10]

Sample shape: (205701, 8)
Skipped files: 1


['laguardia_weather_2022_tavg.csv']

In [7]:
out_path = "reduced_data_to_plot_7.csv"
small.to_csv(out_path, index=False)

print("File size (MB):", os.path.getsize(out_path) / (1024 * 1024))

File size (MB): 24.80920124053955
